In [2]:
from box import ConfigBox

In [3]:
d={'key':'value','key1':'value1'}

In [4]:
d.key

AttributeError: 'dict' object has no attribute 'key'

In [ ]:
# if we waant to call the above value using key like "d.key"
# it will give error. hence we use config box



In [5]:

d2=ConfigBox({'key':'value','key1':'value1'})

In [6]:
d2.key

'value'

creating data class (instad of creating python class where we need to define
the self keyword. we don't need to use in data class)

In [5]:
#update the entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    
    
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict

In [7]:
from mlproject.constant import *
from mlproject.utils.common import read_yaml_file,create_directories

In [14]:
#update the configuration manager in src config
class ConfigurationManager:
    def __init__(self,
                config_filepath=config_file_path,
                params_filepath=config_file_path,
                schema_filepath=config_file_path):
        self.config=read_yaml_file(config_filepath)
        self.params=read_yaml_file(params_filepath)
        self.schema=read_yaml_file(schema_filepath)
        
        create_directories([self.config.artifacts_root])  # this artifacts root name mentioned in the config.yaml
        
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        
        create_directories([self.config.artifacts_root])
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        
        return data_ingestion_config
        

In [15]:
# update the components
import os 
import urllib.request as request
from mlproject import logger
import zipfile

class DataIngestion:
    def __init__(self,config=DataIngestionConfig):
        self.config=config
        
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,header=request.urlretrieve(
            url=self.config.source_url,
            filename=self.config.local_data_file)
            logger.info(f"{filename} download! with following info: \n {header}")
        else:
            logger.info(f"file already exists")
            

    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:
try:
    config=ConfigurationManager()  ## reading the yaml file and passing the parameters/values
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'